It is very bad code for parsing, but it worked. I will rewrite it later. Or not. Also, libraries for parsing are not in requirements.txt, because this project is not about parsing.

# url

In [108]:
import time

import json
from tqdm import tqdm

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains

In [6]:
link = 'https://www.lesswrong.com/users/eliezer_yudkowsky?sortedBy=top'

In [7]:
def get_browser(link):
    op = webdriver.ChromeOptions()
    # op.add_argument("headless")
    browser = webdriver.Chrome(options=op)
    browser.get(link)
    return browser

In [8]:
browser = get_browser(link)

In [74]:
for i in range(20):
    print(i)
    try:
        load_more_button = browser.find_element(By.XPATH, '//*[@id="wrapper"]/div[4]/div[2]/div[2]/div[4]/div[3]/aside/a')
        ActionChains(browser).move_to_element(load_more_button).click(load_more_button).perform()
        time.sleep(3)
    except:
        time.sleep(3)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19


In [79]:
def get_inner_table(row):
    try:
        row = row.find_element(
            by=By.XPATH, 
            value='./div[@class="LWPostsItem-root LWPostsItem-background LWPostsItem-bottomBorder"]'
        )
    except:
        row = row.find_element(
            by=By.XPATH, 
            value='./div[@class="LWPostsItem-root LWPostsItem-background"]'
        )
    
    row = row.find_element(
        by=By.XPATH, 
        value='./div[@class="LWPostsItem-postsItem LWPostsItem-withGrayHover"]'
    )
    return row

In [80]:
table = browser.find_element(By.XPATH, '//*[@id="wrapper"]/div[4]/div[2]/div[2]/div[4]/div[3]/div')

In [81]:
table_rows = table.find_elements(by=By.CLASS_NAME, value='LWPostsItem-row')
table_rows = [get_inner_table(row) for row in table_rows]
len(table_rows)

510

In [98]:
# title
table_rows[0].find_element(
    by=By.XPATH, 
    value='./span[@class="LWPostsItem-title"]/span/span[@class="PostsTitle-root"]/span[@class="PostsTitle-eaTitleDesktopEllipsis"]'
).text

'AGI Ruin: A List of Lethalities'

In [99]:
# url
table_rows[0].find_element(
    by=By.XPATH, 
    value='./span[@class="LWPostsItem-title"]/span/span[@class="PostsTitle-root"]/span[@class="PostsTitle-eaTitleDesktopEllipsis"]/a'
).get_attribute('href')

'https://www.lesswrong.com/posts/uMQ3cqWDPHhjtiesc/agi-ruin-a-list-of-lethalities'

In [84]:
# karma
table_rows[0].find_element(
    by=By.XPATH, 
    value='./span[@class="Typography-root Typography-body2 PostsItem2MetaInfo-metaInfo LWPostsItem-karma"]'
).text

'802'

In [102]:
# comments
table_rows[0].find_element(
    by=By.XPATH, 
    value='./div[@class="LWPostsItem-commentsIcon"]'
).text

'660'

In [104]:
result = {}
for cnt, row in enumerate(tqdm(table_rows)):
    title = row.find_element(
        by=By.XPATH, 
        value='./span[@class="LWPostsItem-title"]/span/span[@class="PostsTitle-root"]/span[@class="PostsTitle-eaTitleDesktopEllipsis"]'
    ).text
    url = row.find_element(
        by=By.XPATH, 
        value='./span[@class="LWPostsItem-title"]/span/span[@class="PostsTitle-root"]/span[@class="PostsTitle-eaTitleDesktopEllipsis"]/a'
    ).get_attribute('href')
    karma = row.find_element(
        by=By.XPATH, 
        value='./span[@class="Typography-root Typography-body2 PostsItem2MetaInfo-metaInfo LWPostsItem-karma"]'
    ).text
    karma = int(karma)
    n_comments = row.find_element(
        by=By.XPATH, 
        value='./div[@class="LWPostsItem-commentsIcon"]'
    ).text
    n_comments = int(n_comments)

    result[cnt] = {}
    result[cnt]['title'] = title
    result[cnt]['url'] = url
    result[cnt]['karma'] = karma
    result[cnt]['n_comments'] = n_comments

100%|██████████| 510/510 [01:07<00:00,  7.55it/s]


In [106]:
result[0]

{'title': 'AGI Ruin: A List of Lethalities',
 'url': 'https://www.lesswrong.com/posts/uMQ3cqWDPHhjtiesc/agi-ruin-a-list-of-lethalities',
 'karma': 802,
 'n_comments': 660}

In [109]:
with open('title_url.json', "w") as fp:
    json.dump(result, fp, sort_keys=True, indent=4, ensure_ascii=False)

# text

In [144]:
from datetime import datetime
import time

import json
from tqdm import tqdm

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains

In [133]:
DEBUG_URL = 'https://www.lesswrong.com/posts/uXn3LyA8eNqpvdoZw/preface'
DEBUG_URL = 'https://www.lesswrong.com/posts/KYuR2HcWPEmXZqMZs/eliezer-yudkowsky-s-shortform'
DEBUG_URL = 'https://www.lesswrong.com/posts/uMQ3cqWDPHhjtiesc/agi-ruin-a-list-of-lethalities'

In [145]:
def date_into_ds(date):
    date = datetime.strptime(date, "%dth %b %Y")
    date = date.strftime("%Y%m%d")
    return date

In [125]:
def get_empty_browser():
    op = webdriver.ChromeOptions()
    # op.add_argument("headless")
    browser = webdriver.Chrome(options=op)
    return browser

In [126]:
browser = get_empty_browser()

In [134]:
browser.get(DEBUG_URL)

In [135]:
# text
browser.find_element(
    by=By.XPATH, 
    value='.//*[@class="commentOnSelection"]'
).text

'Preamble:\n(If you\'re already familiar with all basics and don\'t want any preamble, skip ahead to Section B for technical difficulties of alignment proper.)\nI have several times failed to write up a well-organized list of reasons why AGI will kill you.  People come in with different ideas about why AGI would be survivable, and want to hear different obviously key points addressed first.  Some fraction of those people are loudly upset with me if the obviously most important points aren\'t addressed immediately, and I address different points first instead.\nHaving failed to solve this problem in any good way, I now give up and solve it poorly with a poorly organized list of individual rants.  I\'m not particularly happy with this list; the alternative was publishing nothing, and publishing this seems marginally more dignified.\nThree points about the general subject matter of discussion here, numbered so as not to conflict with the list of lethalities:\n-3.  I\'m assuming you are al

In [146]:
# date
date = browser.find_element(
    by=By.XPATH, 
    value='.//*[@class="PostsPageDate-date"]'
).text
print(date)
date = date_into_ds(date)
date

6th Jun 2022


'20220606'

In [210]:
# for start
# with open('title_url.json') as json_file:
#     urls = json.load(json_file)

# for continue
with open('title_url_text.json') as json_file:
    urls = json.load(json_file)

In [209]:
browser = get_empty_browser()

In [211]:
for key, data in tqdm(urls.items()):
    if data.get('text'):
        continue
    
    browser.get(data['url'])
    date = browser.find_element(
        by=By.XPATH, 
        value='.//*[@class="PostsPageDate-date"]'
    ).text
    urls[key]['date'] = date

    text = browser.find_element(
        by=By.XPATH, 
        value='.//*[@class="commentOnSelection"]'
    ).text
    urls[key]['text'] = text

    time.sleep(1)

100%|██████████| 510/510 [10:07<00:00,  1.19s/it]


In [215]:
urls['116']

{'date': '13th Oct 2012',
 'karma': 134,
 'n_comments': 284,
 'text': 'Suppose a general-population survey shows that people who exercise less, weigh more. You don\'t have any known direction of time in the data - you don\'t know which came first, the increased weight or the diminished exercise. And you didn\'t randomly assign half the population to exercise less; you just surveyed an existing population.\nThe statisticians who discovered causality were trying to find a way to distinguish, within survey data, the direction of cause and effect - whether, as common sense would have it, more obese people exercise less because they find physical activity less rewarding; or whether, as in the virtue theory of metabolism, lack of exercise actually causes weight gain due to divine punishment for the sin of sloth.\n vs. \nThe usual way to resolve this sort of question is by randomized intervention. If you randomly assign half your experimental subjects to exercise more, and afterward the incre

In [212]:
cnt = 0
for key, data in urls.items():
    if data.get('text'):
        cnt += 1
cnt, cnt / len(urls)

(510, 1.0)

In [213]:
with open('title_url_text.json', "w") as fp:
    json.dump(urls, fp, sort_keys=True, indent=4, ensure_ascii=False)